In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
from  pandas_profiling import ProfileReport

In [ ]:
!pip install xgboost --upgrade
from xgboost import XGBClassifier
!pip install category_encoders
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder
! pip install optuna
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [ ]:
import xgboost as xgb
import pandas as pd
import sqlite3
import os



DATABASE_PATH = os.path.join(os.getcwd(), 'main.db')
conn = sqlite3.connect(DATABASE_PATH)

df = pd.read_sql_query(
"""
Select * FROM
(
SELECT * FROM 출전표_상세정보[출상정]
LEFT OUTER JOIN 출전_등록말_정보[출마정]
ON 출상정.MatchingPeriod =출마정.MatchingPeriod AND 출상정.hrNo =출마정.hrNo
)[출상정2]
LEFT OUTER JOIN 경주성적정보[경성정]
ON 출상정2.MatchingPeriod =경성정.MatchingPeriod AND 출상정2.hrNo =경성정.hrNo
LEFT OUTER JOIN 기수기간별성적비교[기성적]
ON 출상정2.MatchingPeriod =기성적.MatchingPeriod기수 AND 출상정2.jkNo =기성적.jkNo기수
LEFT OUTER JOIN 마주기간별성적비교[주성적]
ON 출상정2.MatchingPeriod =주성적.MatchingPeriod마주 AND 출상정2.owNo =주성적.owNo마주
LEFT OUTER JOIN 조교사기간별전적비교[조성적]
ON 출상정2.MatchingPeriod =조성적.MatchingPeriod조교 AND 출상정2.trNo =조성적.trNo조교;

""", conn)

In [ ]:

report = ProfileReport(df, minimal=True)

In [ ]:
descriptions = report.get_description()['alerts']

In [ ]:
for d in descriptions:
  try:
    if d.alert_type_name == 'Constant':
      df = df.drop(labels=d.column_name, axis=1)
    if d.alert_type_name == 'High Cardinality':
      if d.__dict__['values']['n_distinct'] > 50:
        df = df.drop(labels=d.column_name, axis=1)
    if d.alert_type_name == 'Imbalance':
      if d.__dict__['values']['imbalance'] > 0.4:
        df = df.drop(labels=d.column_name, axis=1)
    if d.alert_type_name == 'Missing':
      if d.__dict__['values']['p_missing']>0.35:
        df = df.drop(labels=d.column_name, axis=1)
    if d.alert_type_name == 'Zeros':
      if d.values['p_zeros']>0.45:
        df = df.drop(labels=d.column_name, axis=1)
  except:
    pass

In [ ]:
df = df.replace(to_replace=['', None], value=pd.np.nan)
df = df = df.replace(to_replace='^[A-Z0-9]{6}$', value=pd.np.nan, regex=True)

<ipython-input-8-8143cca0324a>:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df = df.replace(to_replace=['', None], value=pd.np.nan)
<ipython-input-8-8143cca0324a>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df = df = df.replace(to_replace='^[A-Z0-9]{6}$', value=pd.np.nan, regex=True)


In [ ]:
for col in df.select_dtypes(include='object').columns:
  try:
    df[col] = df[col].astype(float)
  except:
    pass

In [ ]:
object_columns = df.select_dtypes(include='object')

object_columns = object_columns.drop(columns=['sex'])

df = df.drop(columns=object_columns.columns)


In [ ]:
df = df.T.drop_duplicates(keep='first').T

In [ ]:
for col in df.select_dtypes(include='object').columns:
  try:
    df[col] = df[col].astype(float)
  except:
    pass

In [ ]:
OrdinalMap = [
              {'col': 'sex',
               'mapping': {'수': 3, '거': 2, '암': 1}}
             ]

In [ ]:
pipe = make_pipeline(
    OrdinalEncoder(mapping = OrdinalMap),
    SimpleImputer(strategy='most_frequent')
)

In [ ]:
df_data = pipe.fit_transform(df)
df = pd.DataFrame(df_data, columns = df.columns)
df = df.rename(columns={'rcDate': 'MatchingPeriod'})

In [ ]:
df['MatchingPeriod'] = df['MatchingPeriod'].astype(int)
df['rcNo'] = df['rcNo'].astype(int)

In [ ]:
df.to_sql('totaltable', conn, if_exists='replace')

In [ ]:
conn.close()

In [ ]:
s=df.groupby('MatchingPeriod').ngroup()
s >= s.quantile(q=0.85)

Test=df.loc[s >= s.quantile(q=0.85)].copy()
Train=df.drop(Test.index)

s=Train.groupby('MatchingPeriod').ngroup()
s >= s.quantile(q=0.75)

Val=Train.loc[s >= s.quantile(q=0.75)].copy()
Train=Train.drop(Val.index)

In [ ]:
from itertools import groupby

X_train=Train.drop(['MatchingPeriod', 'ord', 'rcNo'], axis=1)
y_train = Train['ord']
g_train1=Train.apply(lambda row: str(int(row['MatchingPeriod'])) + format(int(row['rcNo']), '02d'), axis=1)
g_train=[len(list(group)) for key, group in groupby(g_train1)]

X_val=Val.drop(['MatchingPeriod', 'ord', 'rcNo'], axis=1)
y_val = Val['ord']
g_val1=Val.apply(lambda row: str(int(row['MatchingPeriod'])) + format(int(row['rcNo']), '02d'), axis=1)
g_val=[len(list(group)) for key, group in groupby(g_val1)]

X_test=Test.drop(['MatchingPeriod', 'ord', 'rcNo'], axis=1)
y_test = Test['ord']
g_test1=Test.apply(lambda row: str(int(row['MatchingPeriod'])) + format(int(row['rcNo']), '02d'), axis=1)
g_test=[len(list(group)) for key, group in groupby(g_test1)]

In [ ]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.metrics import mean_squared_error


def objectiveXGB(trial: Trial, X, y, g_1, g, X1, y1, g1_1, g1, X2, y2, g2_1, g2):
    param = {
        'tree_method' : 'gpu_hist',
        'booster':'gbtree',
        'objective':'rank:pairwise',
        'eta':0.05,

        "n_estimators" : trial.suggest_int('n_estimators', 1000, 4000),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 300),
        'gamma':trial.suggest_int('gamma', 1, 3),
        'learning_rate': 0.01,
        'colsample_bytree':trial.suggest_float('colsample_bytree',0.5, 1),
        'nthread' : -1,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.6,0.7,0.8] ),
        'random_state': 42
    }
    ## XGBRegressor를 예시로 사용하였습니다.
    model = xgb.XGBRanker(**param)
    model.fit(X, y, g, verbose=True)

    ## RMSE으로 Loss 계산

    predict = model.predict(X1,g1)
    result = pd.DataFrame({'predict': predict, 'g' : g1_1})
    score1 = mean_squared_error(y1, result.groupby('g')['predict'].rank())

    predict = model.predict(X2,g2)
    result = pd.DataFrame({'predict': predict, 'g' : g2_1})
    score2 = mean_squared_error(y2, result.groupby('g')['predict'].rank())

    return max(score1, score2)


# direction : score 값을 최대 또는 최소로 하는 방향으로 지정
study = optuna.create_study(direction='minimize',sampler=TPESampler())

# n_trials : 시도 횟수 (미 입력시 Key interrupt가 있을 때까지 무한 반복)
study.optimize(lambda trial : objectiveXGB(trial, X_train,  y_train,g_train1, g_train, X_val, y_val, g_val1, g_val, X_test, y_test, g_test1, g_test), n_trials=150)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))


In [ ]:
param=study.best_trial.params

model = xgb.XGBRanker(**param)
model.fit(X_train,  y_train, g_train)

XGBRanker(alpha=2.316273772698475, base_score=0.5, booster='gbtree',
          callbacks=None, colsample_bylevel=1, colsample_bynode=1,
          colsample_bytree=0.5499271553383916, early_stopping_rounds=None,
          enable_categorical=False, eval_metric=None, feature_types=None,
          gamma=2, gpu_id=-1, grow_policy='depthwise', importance_type=None,
          interaction_constraints='', lambda=3.3327441503094337,
          learning_rate=0.300000012, max_bin=256, max_cat_threshold=64,
          max_cat_to_onehot=4, max_delta_step=0, max_depth=14, max_leaves=0,
          min_child_weight=1, missing=nan, monotone_constraints='()',
          n_estimators=3746, n_jobs=0, num_parallel_tree=1, ...)

In [ ]:
model.save_model("model.model")

In [ ]:
df

In [ ]:
X_df=df.drop(['MatchingPeriod', 'ord', 'rcNo'], axis=1)
y_df = df['ord']
g_df1=df.apply(lambda row: str(int(row['MatchingPeriod'])) + format(int(row['rcNo']), '02d'), axis=1)
g_df=[len(list(group)) for key, group in groupby(g_df1)]

In [ ]:
predict = model.predict(X_df,g_df)
result = pd.DataFrame({'y': y_df, 'predict': predict, 'g' : g_df1})

result = result.assign(predict2 = result.groupby('g')['predict'].rank())

In [ ]:
DATABASE_PATH = os.path.join(os.getcwd(), 'main.db')
conn = sqlite3.connect(DATABASE_PATH)
result.to_sql('resulttable', conn, if_exists='replace')

In [ ]:
conn.close()

In [ ]:
df[['ord']]

,ord
0,7.0
1,3.0
2,5.0
3,11.0
4,8.0
...,...
46219,6.0
46220,7.0
46221,8.0
46222,92.0


In [ ]:
count = df[['owNo']].isnull().sum()

In [ ]:
df[['owNo']].drop(count[count == count.max()].index[0], axis=1)

""
0
1
2
3
4
...
46219
46220
46221
46222


In [ ]:
!pip install snowflake-connector-python

In [ ]:
import snowflake.connector

In [ ]:
#!/usr/bin/env python
import snowflake.connector

# Gets the version
cnx = snowflake.connector.connect(
    user='tjfdnjfch',
    password='Luna0327*',
    account='vyytohq-ru80624',
    database='PROJECT3',
    schema='PUBLIC'
    )

In [ ]:
from snowflake.connector.pandas_tools import write_pandas
import pandas
from snowflake.connector.pandas_tools import pd_writer

In [ ]:
import snowflake.connector

# Connect to Snowflake
cnx = snowflake.connector.connect(
    user='tjfdnjfch',
    password='Luna0327*',
    account='vyytohq-ru80624',
    database='PROJECT3',
    schema='PUBLIC'
)

# Create a cursor
cursor = cnx.cursor()

df=pd.read_csv('/content/경주성적정보.csv')
cursor.execute("COPY INTO 경주성적정보 FROM %s", (df,))

# Close the cursor and connection
cursor.close()
cnx.close()


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (24,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


ProgrammingError: ignored

In [ ]:
import sqlite3
import snowflake.connector

# Connect to the SQLite database
conn = sqlite3.connect("main.db")
cursor = conn.cursor()

# Retrieve the data from the SQLite database
cursor.execute("SELECT * FROM 경주성적정보")
results = cursor.fetchall()

cols =str(tuple(temp.columns)).replace("'","")
vals = str(tuple(['?']*len(temp.columns))).replace("'","")

executer = f"INSERT INTO mytable {cols} VALUES {vals}"






# Connect to Snowflake
cnx = snowflake.connector.connect(
    user='tjfdnjfch',
    password='Luna0327*',
    account='vyytohq-ru80624',
    database='PROJECT3',
    schema='PUBLIC'
    )

# Create a cursor in Snowflake
cursor = cnx.cursor()

# Insert the data into a table in Snowflake
for row in results:
  cursor.execute("INSERT INTO mytable (col1, col2, col3) VALUES (?, ?, ?)", row)

# Commit the transaction
cnx.commit()

# Close the connections
conn.close()
cnx.close()


DatabaseError: ignored

In [ ]:
temp = pd.read_csv('경주성적정보.csv')
tuple(temp.columns).__str__()

"('age', 'ageCond', 'birthday', 'budam', 'buga1', 'buga2', 'buga3', 'chaksun1', 'chaksun2', 'chaksun3', 'chaksun4', 'chaksun5', 'chulNo', 'diffUnit', 'g2f', 'g3f_4c', 'g4f_3c', 'g6f_2c', 'g8f_1c', 'hrName', 'hrNo', 'hrTool', 'ilsu', 'jkName', 'jkNo', 'meet', 'name', 'ord', 'ordBigo', 'ordG1f', 'ordS1f', 'owName', 'owNo', 'plcOdds', 'prizeCond', 'rank', 'rankRise', 'rating', 'rcDate', 'rcDay', 'rcDist', 'rcName', 'rcNo', 'rcTime', 'rcTimeG1f', 'rcTimeG2f', 'rcTimeG3f', 'rcTimeS1f', 'rcTime_1c', 'rcTime_2c', 'rcTime_3c', 'rcTime_400', 'rcTime_4c', 'sex', 'trName', 'trNo', 'track', 'weather', 'wgBudam', 'wgBudamBigo', 'wgHr', 'wgJk', 'winOdds', 'MatchingPeriod')"

In [ ]:
cols =str(tuple(temp.columns)).replace("'","")
vals = str(tuple(['?']*len(temp.columns))).replace("'","")

executer = f"INSERT INTO mytable {cols} VALUES {vals}"

'INSERT INTO mytable (age, ageCond, birthday, budam, buga1, buga2, buga3, chaksun1, chaksun2, chaksun3, chaksun4, chaksun5, chulNo, diffUnit, g2f, g3f_4c, g4f_3c, g6f_2c, g8f_1c, hrName, hrNo, hrTool, ilsu, jkName, jkNo, meet, name, ord, ordBigo, ordG1f, ordS1f, owName, owNo, plcOdds, prizeCond, rank, rankRise, rating, rcDate, rcDay, rcDist, rcName, rcNo, rcTime, rcTimeG1f, rcTimeG2f, rcTimeG3f, rcTimeS1f, rcTime_1c, rcTime_2c, rcTime_3c, rcTime_400, rcTime_4c, sex, trName, trNo, track, weather, wgBudam, wgBudamBigo, wgHr, wgJk, winOdds, MatchingPeriod) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'

In [ ]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3, 4, 1, 2, 3, 4], 'B': [4, 5, 6, 7, 7, 6, 5, 4], 'C': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b']})

# Rank the rows within groups based on the values in column 'B'
df['rank'] = df.groupby('C')['B'].rank()

print(df)


   A  B  C  rank
0  1  4  a   1.0
1  2  5  a   2.0
2  3  6  a   3.0
3  4  7  a   4.0
4  1  7  b   4.0
5  2  6  b   3.0
6  3  5  b   2.0
7  4  4  b   1.0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
